## A Hands-on Workshop series in Machine Learning
### Session 7: Introduction to Recurrent Neural Networks (RNN)
#### Instructor: Aashita Kesarwani

### Background and revision

In the previous sessions, we learned about the Bag-Of-Words (BOW) method for vectorization of input sentences. The input/feature vectors were of the length of the pre-defined vocabulary and we used either simple frequencies or TF-IDF frequencies for converting input sentences to numerical feature vectors. The two main drawbacks of this method are:
* The dimension for input/feature vectors is too high. If we use feedforward neural networks, the number of input nodes is too large, so consequently the nodes in subsequent layers would need to be proportionally high for not losing a lot of information. This would make training process computationally expensive.
* The ordering of words in the input sentences is completely lost. We know that a lot of information in our languages is encoded in the order of words, so we should think of using an architecture that would take it into account.

To remedy the above, we can
* Set a certain length for input sequences, say 20 words
* Process all the sentences in the training dataset so they consist of exactly 20 words, either by padding with null words or chopping off longer sentences
* Create a vocabulary of all words in the training data and assign them hashes (numbers)
* Vectorize the input sequences by replacing the words with their hashes in the vocabulary

The feed forward neural network with the above input might be able to capture information about the order of words in the sentence but it would need to learn all rules of the language separately at each point. For example, it would treat the following two sentences differently - “I went to Hawaii in 2018” and “In 2018, I went to Hawaii.” The relevant information about when the narrator went to Hawaii is present in 6th and 2nd position respectively. Many more training examples would be needed to train the network to extract the relevant information as the network needs to learn the same rules separately for all positions in the input layer.

Recurrent Neural Networks (RNN) are designed and optimized for sequential data. They are predominantly applied in the fields of natual language processing and speech recognition but can be useful for other time-series based data as well. 

### Recurrent Neural Networks (RNN)

* RNN takes an input of sequence, denoted as $x^{(1)}, x^{(2)}, \dots, x^{(t)}, \dots$, instead of a single input vector as seen previously.
* It can handle input sequences of any length.
* The information from the time step $t-1$ is passed on as input to the next time step $t$.

<img src="./images/Fig0.png" width="500" height="70" />


In mathematics, we often study dynamical systems that involves recurrence relations:

$$s^{(t)} = f(s^{(t-1)}; \theta)$$

where $s^{(t)}$ is the state of system at time $t$.

These dynamical systems can also involve input signals $x^{(t)}$ at each step:

$$s^{(t)} = f(s^{(t-1)}, x^{(t)} ; \theta)$$

Such a recurrence relation is involved in the architecture of most RNN (though the architecture can vary greatly as discussed below). The hidden nodes in RNN at time $t$ can be defined as:

$$h^{(t)} = f(h^{(t-1)}, x^{(t)} ; \theta)$$

Here, the hidden state $h^{(t)}$ defined in terms of $h^{(t-1)}$ can be unfolded in the following manner, for say $t=3$:

\begin{align}
 h^{(3)} &= f(h^{(2)}, x^{(3)}; \theta) \\ 
 &= f(f(h^{(1)}, x^{(2)}; \theta), x^{(3)}; \theta) \\
 &= f(f(f(h^{(0)}, x^{(1)}; \theta), x^{(2)}; \theta), x^{(3)}; \theta) \\
 &= g^{(3)}(x^{(1)}, x^{(2)}, x^{(3)})
\end{align}

Thus, the unfolded recurrence can also be represented by a function $g^{(t)}$ as:

\begin{align}
 h^{(t)} &= f(h^{(t-1)}, x^{(t)}; \theta) \\ 
 &= g^{(t)}(x^{(1)}, \dots, x^{(t-1)}, x^{(t)})
\end{align}

Advantages of using the model $h^{(t)} = f(h^{(t-1)}, x^{(t)} ; \theta)$ over $h^{(t)} = g^{(t)}(x^{(1)}, \dots, x^{(t-1)}, x^{(t)})$:
* The function $g^{(t)}$ can be a different function at each time step but the ***same*** transition function $f$ with the same parameters $\theta$ can be used to define hidden state in terms of $h^{(t-1)}$ and input $x^{(t)}$ for every time step. 
* The input sequences can be of variable length but the input for the model is always the same size consisting of $h^{(t-1)}$ and input $x^{(t)}$.

Thus, a single shared model $f$ can be learned that will generalize to sequences of any length and will eliminate the need to learn a separate model $g^{(t)}$ for each time step. This parameter sharing also allows us to train the model with far fewer training examples.

As the RNN involve function compositions multiple times, the composite function can result in very high non-linear behaviour.

Some common types of RNN architecture:
### 1. Output at each step and recurrent connections between hidden units 

<img src="./images/Fig3.png" width="400" height="70" />

The equations for the simple RNN architecture:

\begin{align}
 a^{(t)} &= b + Wh^{(t-1)} + U x^{(t)} \\ 
 h^{(t)} &= \tanh (a^{(t)})\\
 o^{(t)} &= c + Vh^{(t)}\\
 \hat{y}^{(t)} &= \text{softmax} (o^{(t)})\\
\end{align}

The activation function for the hidden unit is hyperbolic tangent which is simply sigmoid function scaled and translated along y-axis.

<img src="https://www.oreilly.com/library/view/mastering-machine-learning/9781788621113/assets/e96a4fab-73ae-47fa-81e0-5a4780016a07.png" width="500" height="100" />

Cost function, $L$, is the sum of negative loglikelihood of $y^{(t)}$ given $x^{(1)}, \dots, x^{(t-1)}, x^{(t)}$

$$ L = \sum_t L^{(t)} = - \sum_t \log p \big(y^{(t)} | x^{(1)}, \dots, x^{(t-1)}, x^{(t)} \big)$$


### 2. Output at each step and recurrent connections from output at one time step to hidden units at the next

This is a slight modification of the above architecture where the recurrent connection goes from from output at one time step to hidden units at the next instead of connecting the subsequent hidden units. The output is optimized for correct prediction instead of passing on the relevant background information to the next time step. As output is the only information that is allowed to pass on to the next unit, this RNN is less powerful than the above one. As an advantage, this RNN is far easier to train and allows parallelization using teacher forcing explained below.

<img src="./images/Fig4.png" width="400" height="70" />

### 3. A single output at the end and recurrent connections between hidden units
When the output is only needed at the end after reading the entire input sequence, this architecture is used.

<img src="./images/Fig5.png" width="320" height="70" />

Some applications for a single output:
* Sentiment analysis (Text classification)
* Sentence completion (guessing the next word in the sequence)


 #### Back-propagation through time (BPTT):
The forward propagation moves from left-to-right along the time steps whereas the backpropagation moves in the opposite direction from right-to-left. The computations for weight updates in the backpropagation cannot be parallelized for different time steps as the forward propagation is sequential and hence, the computations cannot be done for time step $t=\tau$ unless they are done for all time steps upto $t=\tau-1$. Thus, the RNNs can be powerful but very expensive to train.

####  Teacher forcing
The model of the second type that have recurrent connections from output at one time step to hidden units at the next can be trained using teacher forcing. Instead of using the predicted output from the previous time step, the true/targeted output is used. This makes it possible to parallelize the backpropagation as each time step can be conssidered as an independent unit. It is no longer necessary that the computations are completed for the previous time steps before calculation the gradient for the current one. 

<img src="./images/Fig6.png" width="400" height="70" />

While making predictions on unseen data using the tranined RNN, the predicted output from the previous time step is used.

### The challenge of Long-term dependencies

When we read and try to understand a sentence, we do not only pay attention to the current word, but we also keep the context in mind from what we read previously in the text. This dependence on previous words to understand the context for the current word is called the dependency. The RNN can learn short-term dependencies much better than long-term dependencies, meaning it remembers the context from the neighbouring previous words better but keep losing the information as we move along the sentence.

It can be addressed in the simple RNN by adding
* Clipping gradients: providing a minimum and maximum limit to the gradients to address vanishing/exploding gradients
* Adding/removing skip connections thru time: add connection from distant past to present and/or remove connection from consecutive units

LSTM and GRU are slight modifications to simple RNN to address the problem of long-term dependencies. They improve the computational time and performance to some extent. The breakthrough in the performance of deep learning for NLP came from the introduction of transformer models that use attention mechanism (from the [Attention is all you need](https://arxiv.org/abs/1706.03762) paper) instead of recurrent connections across time steps. Transformer based models such as GPT, GPT-2, and GPT-3 by OpenAI, Google's BERT and many modifications of these algorithms have shown remarkable results in most NLP tasks. Transfer learning can be used for fine-tuning pre-trained transformer models freely available online and applying them for various specific tasks in several different languages. HuggingFace and Papers with Code are two excellent resources along with Kaggle.

### Encoder-decoder or sequence-to-sequence models
* Encoder RNN reads input sequence, called “context” and produces a representation of the context, say C 
* Final hidden state of encoder RNN is used to compute the (generally) fixed-length context variable C, which represents a semantic summary of the input sequence
* Context variable C is Input to the Decoder RNN
* Decoder RNN generates output sequence 

<img src="https://miro.medium.com/max/1400/1*1JcHGUU7rFgtXC_mydUA_Q.jpeg" width="500" height="70" />

The encoder and decoder can also be LSTM/GRU units instead of simple RNN units. The attention-based encoder-decoder or sequence-to-sequence models, also known as transformers, have different architecture involving an attention layer.

#### Further reading
* Reference textbook for learning more about RNN: https://www.deeplearningbook.org/contents/rnn.html
* A wonderful blog for understanding LSTM by Christopher Olah: https://colah.github.io/posts/2015-08-Understanding-LSTMs/
* The Illustrated Transformers blog: https://jalammar.github.io/illustrated-transformer/

#### Acknowledgement
Some content and diagrams are taken from the online book: https://www.deeplearningbook.org/contents/rnn.html
